In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import csv

class layers():
    def __init__(self,neu,r,c):
        self.neu=int(neu)
        self.wt=np.random.randn(r,c)
        self.a=np.zeros((self.neu,1),dtype=np.float64)
        self.z=np.zeros((self.neu,1),dtype=np.float64)
        self.error=np.zeros((self.neu,1),dtype=np.float64)
        self.temp=np.zeros((r,c),dtype=np.float64)
        self.bias=np.random.randn(self.neu,1)
        self.temp_bias=np.zeros((self.neu,1),dtype=np.float64)

class network:
    def __init__(self,data_in,data_out,layers_list,l_rate=0.05):
        self.x=np.array(data_in)/100
        self.y=np.array(data_out)    
        xx=np.array_split(self.x.T,2)
        self.xt=xx[0].T
        self.xc=xx[1].T
        yy=np.array_split(self.y.T,2)
        self.yt=yy[0].T
        self.yc=yy[1].T 
        self.l=int(len(li))
        self.ly=layers_list
        self.l_rate= float(l_rate)
        try:
            self.n=self.xt.shape[1]
        except:
            self.n=self.xt.shape[0]
        self.m=int(self.n/100)
        self.x_split=np.array_split(self.xt.T,self.m)
        self.y_split=np.array_split(self.yt.T,self.m)
     
    def sigm(self,x):
        return 1/(1+np.e**-x)
    def der_sigm(self,x):
        return sigm(x)/(1-sigm(x))
    def forward(self):
        for i in range(self.l):
            if i==0:
                self.ly[i].z=np.dot(self.ly[i].wt,self.xt)+self.ly[i].bias
                self.ly[i].a=self.sigm( self.ly[i].z)
            else:
                self.ly[i].z=np.dot(self.ly[i].wt,self.ly[i-1].a)+self.ly[i].bias
                self.ly[i].a=self.sigm(self.ly[i].z)
    
    def cost(self):
        return np.sum((np.dot(self.yt,np.log(self.ly[-1].a).T))+ np.dot( 1-self.yt,np.log(1-self.ly[-1].a).T ))
    
    def backprop(self):
        for i in range(self.l-1, -1, -1):
            if i!=self.l-1:
                self.ly[i].error=np.dot(self.ly[i+1].wt.T, self.ly[i+1].a)
            else:
                self.ly[i].error=self.ly[i].a-self.yt
    def update(self):
        for i in range(self.l-1, -1, -1):
            if i!=0:
                self.ly[i].temp= np.dot(self.ly[i].error,self.ly[i-1].a.T)
            else:
                self.ly[i].temp= np.dot(self.ly[i].error,self.xt.T)
                
            self.ly[i].temp_bias= np.mean(self.ly[i].error, axis=1)
            
        for i in range(self.l-1, -1, -1):
            self.ly[i].wt=self.ly[i].wt-self.ly[i].temp*self.l_rate
            self.ly[i].bias=self.ly[i].bias-np.mean(self.ly[i].temp_bias, axis=1)*self.l_rate


    def train(self,epoch):
        try:
            epoch=int(int(epoch)/self.m)
            r=int(self.m)
        except:
            epoch=int(epoch)
            r=int(1)
        for i in range(int(epoch)):
            for j in range(int(r)):
                self.xt=self.x_split[i].T
                self.yt=self.y_split[i].T
                self.forward()
                self.cost()
                self.backprop()
                self.update()
    def implement(self,x_data):            # just one sample at a time
        x_data=np.array(x_data)/100
        l=int(x_data.shape[0])
        x_data=np.expand_dims(x_data,0)
        x_data=x_data.T
        self.xi=x_data
        for i in range(self.l):
            if i==0:
                self.ly[i].z=np.dot(self.ly[i].wt,x_data)+self.ly[i].bias
                self.ly[i].a=self.sigm( self.ly[i].z)
            else:
                self.ly[i].z=np.dot(self.ly[i].wt,self.ly[i-1].a)+self.ly[i].bias
                self.ly[i].a=self.sigm(self.ly[i].z)
        print(self.ly[-1].a)




In [3]:

data=pd.read_csv("mnist_train_small.csv",header=None)
data.dropna()
d=np.array(data)
y=d[:,[0]]
x=np.delete(d,0,1)
z=np.eye(10)
yy=np.empty((y.shape[0],10),dtype=np.float64)
for i in range(y.shape[0]):
    for j in range(10):
        if y[i]==j+1:
            yy[i,:]=z[j,:]
        else:
            pass

In [4]:
li=[]
li.append(layers(10,10,784))
li.append(layers(10,10,10))
li.append(layers(10,10,10))
li.append(layers(10,10,10))
li.append(layers(10,10,10))
li.append(layers(10,10,10))


In [18]:
model=network(x.T,y.T,li)
model.train(1)

In [69]:

data=pd.read_csv("mnist_test.csv",header=None)
data.dropna()
d=np.array(data)
y_test=d[:,[0]]
x=np.delete(d,0,1)
z=np.eye(10)
yy=np.empty((y.shape[0],10),dtype=np.float64)
for i in range(y.shape[0]):
    for j in range(10):
        if y[i]==j+1:
            yy[i,:]=z[j,:]
        else:
            pass

array([[0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.]])

In [6]:
model.ly[-1].wt

array([[-1.69977608,  0.56464518, -1.14951219, -0.24750915,  0.45364465,
         0.2505694 ,  1.36297287, -1.28294785,  1.19222762,  1.58164443],
       [-0.7106992 ,  0.23586616, -2.19432919,  0.92257624,  1.66071521,
         1.54689255, -0.81519256, -0.84651815,  0.4791914 ,  1.12772809],
       [-1.31474015, -1.44546283, -0.97104849, -0.36770936,  2.3965853 ,
         2.80317806, -0.60007546,  1.45170488, -0.31887083,  0.48521521],
       [-1.19570865,  0.06999448, -0.35564848, -0.00869066,  0.36711753,
        -1.67356087,  1.09991085,  0.11372201,  1.15688278,  1.1647486 ],
       [-1.01808836, -0.60932253,  0.34583055,  0.58681153, -0.1413575 ,
        -0.50733616,  0.82068992,  0.99313032,  0.27825674, -0.20434223],
       [-0.18420563,  0.87836227,  1.68841015, -0.5882249 , -0.93621592,
         1.38506629, -0.56813936, -0.43810894,  0.15449807,  0.78007476],
       [-1.36139092, -0.63035702,  0.0388052 ,  0.89499131, -0.13334477,
        -0.045995  ,  0.03915251, -1.17930905

In [12]:
np.mean(np.square(model.y-model.ly[-1].a),axis=0)*100

array([3600., 2500., 4900., ...,  400., 8100., 2500.])